In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import eerie

In [2]:
torch.__version__

'2.1.1+cu118'

In [3]:
torch.cuda.is_available()

True

# data

In [6]:
from data_process import *

In [7]:
data_path="./data/Trainingdata_WS_Month_1.0/inputs/"
label_path="./data/Trainingdata_WS_Month_1.0/labels/"


kfold_load_process_save_data(data_path, label_path, is_augment=False, data_shape="6x16", oversampling="", kfold=2, times=5)


6x16 


100%|██████████| 5536/5536 [00:00<00:00, 20278.42it/s]
d:\anaconda3\envs\timm\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


(5536, 96) (5536,)
times 0 fold 0
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 0 fold 1
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 1 fold 0
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 1 fold 1
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 2 fold 0
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 2 fold 1
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 3 fold 0
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 3 fold 1
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 4 fold 0
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)
times 4 fold 1
(2768, 6, 16) (2768,) (2768, 6, 16) (2768,)


In [8]:
data16x6 = np.load("./data/2fold/6x16/T0_fold0_noOversampling_X_train.npy")
data16x6.shape

(2768, 6, 16)

In [9]:
data16x6[0].shape

(6, 16)

# OneDCNN

In [3]:
from model import *

model = OneDCNN(n_channels=64, hidden=128, n_classes=17)
sum([p.numel() for p in model.parameters()])

153873

In [4]:
class OneDCNN(torch.nn.Module):
    def __init__(self, n_channels=16, hidden=16, n_classes=17, eps=2e-5, use_bias=False, dp_rate=0.25):
        super(OneDCNN, self).__init__()

        # Conv Layers
        self.c1 = torch.nn.Conv1d(in_channels=6,              out_channels=n_channels,      kernel_size=8,  stride=1, padding='same', dilation=1, bias=use_bias)
        self.c2 = torch.nn.Conv1d(in_channels=n_channels,     out_channels=n_channels * 2,  kernel_size=4,  stride=1, padding='same', dilation=1, bias=use_bias)
        self.c3 = torch.nn.Conv1d(in_channels=n_channels * 2, out_channels=n_channels * 4,  kernel_size=2,  stride=1, padding='same', dilation=1, bias=use_bias)
        
        # Fully connected
        self.f1 = torch.nn.Linear(in_features=n_channels * 4, out_features=hidden,    bias=True)
        self.f2 = torch.nn.Linear(in_features=hidden,         out_features=hidden,    bias=True)
        self.f3 = torch.nn.Linear(in_features=hidden,         out_features=n_classes, bias=True)

        # BatchNorm Layers
        self.bn1 = torch.nn.BatchNorm1d(num_features=n_channels,      eps=eps)
        self.bn2 = torch.nn.BatchNorm1d(num_features=n_channels * 2,  eps=eps)
        self.bn3 = torch.nn.BatchNorm1d(num_features=n_channels * 4,  eps=eps)

        # Pooling
        self.pool = torch.max_pool1d
        # self.pool = nn.MaxPool1d(kernel_size=3, stride=1, padding='same')

        # DropOut
        self.dropout = torch.nn.Dropout(p=dp_rate)

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                m.weight.data.normal_(0, torch.prod(torch.Tensor(list(m.weight.shape)[1:]))**(-1/2))
                if use_bias: m.bias.data.fill_(0.0)

    def forward(self, x):
        # Conv-layers
        out = self.bn1(torch.relu(self.c1(x)))
        print("1 ", out.shape)
        # out = self.pool(out, kernel_size=8, stride=8, padding=0)
        out = self.pool(out, kernel_size=4, stride=2, padding=1)
        print("1 ", out.shape)
        out = self.bn2(torch.relu(self.c2(out)))
        print("2 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=2, padding=1)
        print("2 ", out.shape)
        out = self.bn3(torch.relu(self.c3(out)))
        print("3 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=4, padding=1)
        print("3 ", out.shape)

        # Fully connected lyrs
        out = out.view(out.size(0), -1)
        out = self.dropout(self.f1(out))
        out = self.dropout(self.f2(out))
        out = self.f3(out)

        return out

In [5]:
model = OneDCNN(n_channels=64, hidden=128, n_classes=17)
sum([p.numel() for p in model.parameters()])

153873

In [6]:
x = torch.ones((128, 6, 16))
y = model(x)
y.shape

1  torch.Size([128, 64, 16])
1  torch.Size([128, 64, 8])
2  torch.Size([128, 128, 8])
2  torch.Size([128, 128, 4])
3  torch.Size([128, 256, 4])
3  torch.Size([128, 256, 1])


d:\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Convolution.cpp:1004.)
  return F.conv1d(input, weight, bias, self.stride,


torch.Size([128, 17])

# EquiOneDCNN

In [10]:
class EquiOneDCNN(torch.nn.Module):
    def __init__(self, n_channels=16, hidden=16, n_classes=17, eps=2e-5, use_bias=False, dp_rate=0.25):
        super(EquiOneDCNN, self).__init__()

        # # G-conv approach
        group = eerie.Group('R1R+')

        # For first layer:
        N_h_RdG = 9
        base = 2
        h_grid_RdG = group.h_grid_global(N_h_RdG, base ** (N_h_RdG - 1))
        print(h_grid_RdG.grid)

        N_h_crop = 3  # <--- TODO: not sure if this is the most optimal though, but it reduces the h_axis nicely to size 1 in the last layer
        base = 2
        h_grid_crop = group.h_grid_global(N_h_crop, base ** (N_h_crop - 1))
        print(h_grid_crop.grid)

        # Conv Layers
        self.c1 = eerie.nn.GConvRdG(group, in_channels=6,              out_channels=n_channels,     kernel_size=7, h_grid=h_grid_RdG,  bias=use_bias, stride=1)
        self.c2 = eerie.nn.GConvGG(group,  in_channels=n_channels,     out_channels=n_channels * 2, kernel_size=3, h_grid=h_grid_crop, bias=use_bias, stride=1, h_crop=True)
        self.c3 = eerie.nn.GConvGG(group,  in_channels=n_channels * 2, out_channels=n_channels * 4, kernel_size=1, h_grid=h_grid_crop, bias=use_bias, stride=1, h_crop=True)
        
        # Fully connected
        self.f1 = torch.nn.Linear(in_features=n_channels * 4 * (N_h_RdG+1)//2, out_features=hidden, bias=True)
        self.f2 = torch.nn.Linear(in_features=hidden, out_features=hidden,    bias=True)
        self.f3 = torch.nn.Linear(in_features=hidden, out_features=n_classes, bias=True)

        # BatchNorm Layers
        self.bn1 = torch.nn.BatchNorm2d(num_features=n_channels,      eps=eps)
        self.bn2 = torch.nn.BatchNorm2d(num_features=n_channels * 2,  eps=eps)
        self.bn3 = torch.nn.BatchNorm2d(num_features=n_channels * 4,  eps=eps)
        
        # Pooling
        self.pool = eerie.functional.max_pooling_R1
        # DropOut
        self.dropout = torch.nn.Dropout(p=dp_rate)

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                m.weight.data.normal_(0, torch.prod(torch.Tensor(list(m.weight.shape)[1:]))**(-1/2))
                if use_bias: m.bias.data.fill_(0.0)

    def forward(self, x):
        # Conv-layers
        # We replace strided convolutions with normal convolutions followed by max pooling.
        # -----
        out = self.c1(x)
        print("1 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=2, padding=1)
        out = self.bn1(torch.relu(out))
        print("1 ", out.shape)
        # -----
        # out = self.pool(out, kernel_size=8, stride=8, padding=0)
        # print("1 ", out.shape)
        # -----
        out = self.c2(out)
        print("2 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=2, padding=1)
        out = self.bn2(torch.relu(out))
        print("2 ", out.shape)
        # -----
        # out = self.pool(out, kernel_size=8, stride=8, padding=0)
        # print("2 ", out.shape)
        # -----
        out = self.c3(out)
        print("3 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=4, padding=1)
        out = self.bn3(torch.relu(out))
        print("3 ", out.shape)
        # # -----
        # out = self.c4(out)
        # out = self.pool(out, kernel_size=2, stride=2, padding=0)
        # out = self.bn4(torch.relu(out))
        # print("4 ", out.shape)
        # # -----
        # out = self.c5(out)
        # out = self.pool(out, kernel_size=2, stride=2, padding=0)
        # out = self.bn5(torch.relu(out))
        # print("5 ", out.shape)
        # # -----
        # # out = self.pool(out, kernel_size=5, stride=5, padding=0)
        # # print("5 ", out.shape)
        # -----
        # Fully connected lyrs
        out = out.view(out.size(0), -1)
        out = self.dropout(self.f1(out))
        out = self.dropout(self.f2(out))
        out = self.f3(out)

        return out

In [11]:
model = EquiOneDCNN(n_channels=128, hidden=128, n_classes=17)
sum([p.numel() for p in model.parameters()])

tensor([[  1.],
        [  2.],
        [  4.],
        [  8.],
        [ 16.],
        [ 32.],
        [ 64.],
        [128.],
        [256.]])
tensor([[1.],
        [2.],
        [4.]])


1041809

In [12]:
x = torch.ones((128, 6, 16))
y = model(x)
y.shape

1  torch.Size([128, 128, 9, 16])
1  torch.Size([128, 128, 9, 8])
2  torch.Size([128, 256, 7, 8])
2  torch.Size([128, 256, 7, 4])
3  torch.Size([128, 512, 5, 4])
3  torch.Size([128, 512, 5, 1])


torch.Size([128, 17])

# ResNet50

In [56]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, heads=4, mhsa=False, resolution=None):
        super(Bottleneck, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        if not mhsa:
            self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1, stride=stride, bias=False)
        else:
            self.conv2 = nn.ModuleList()
            self.conv2.append(MHSA(planes, width=int(resolution[0]), height=int(resolution[1]), heads=heads))
            if stride == 2:
                self.conv2.append(nn.AvgPool2d(2, 2))
            self.conv2 = nn.Sequential(*self.conv2)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000, resolution=(224, 224), heads=4, is_botnet=False):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.resolution = list(resolution)

        self.conv1 = nn.Conv2d(1, 16, kernel_size=7, stride=2, padding=3, bias=False)
        if self.conv1.stride[0] == 2:
            self.resolution[0] /= 2
        if self.conv1.stride[1] == 2:
            self.resolution[1] /= 2
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # for ImageNet
        if self.maxpool.stride == 2:
            self.resolution[0] /= 2
            self.resolution[1] /= 2

        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        if is_botnet == False:
            self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2, heads=heads, mhsa=False)
        else:
            self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=1, heads=heads, mhsa=True)
            
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Dropout(0.3), # All architecture deeper than ResNet-200 dropout_rate: 0.2
            nn.Linear(128 * block.expansion, num_classes)
        )

    def _make_layer(self, block, planes, num_blocks, stride=1, heads=4, mhsa=False):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for idx, stride in enumerate(strides):
            layers.append(block(self.in_planes, planes, stride, heads, mhsa, self.resolution))
            if stride == 2:
                self.resolution[0] /= 2
                self.resolution[1] /= 2
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out) # for ImageNet

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


# def ResNet50(num_classes=17, resolution=(16, 16), heads=4):
#     return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, resolution=resolution, heads=heads, is_botnet=False)

def get_ResNet(num_blocks=[3, 4, 6, 3], num_classes=17, resolution=(16, 16)):
    return ResNet(block=Bottleneck, num_blocks=num_blocks, num_classes=num_classes, resolution=resolution, is_botnet=False)


def get_BoTNet(num_blocks=[3, 4, 6, 3], num_classes=17, resolution=(16, 16), heads=4):
    return ResNet(block=Bottleneck, num_blocks=num_blocks, num_classes=num_classes, resolution=resolution, is_botnet=True, heads=heads)


In [59]:
resnet = get_ResNet(num_blocks=[1,1,1,1], num_classes=17, resolution=(16,16))
sum([p.numel() for p in resnet.parameters()])

514497

In [60]:
x = torch.ones((128, 1, 16, 16))
y = resnet(x)
y.shape

torch.Size([128, 17])

# EquiResNet

In [18]:
class RRPlus_IdentityBlock(torch.nn.Module):
    def __init__(self, group, in_channels, out_channels, kernel_size, h_grid, h_grid_crop, stride, use_bias, eps, override=False):
        super(RRPlus_IdentityBlock, self).__init__()

        # Additional params
        self.diff_size = (in_channels != out_channels)
        self.h_grid = h_grid_crop if self.diff_size else h_grid
        self.crop = h_grid_crop.N
        self.override = override

        # Conv Layers
        if not override:
            self.c1 = eerie.nn.GConvGG(
                group, 
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=kernel_size, 
                h_grid=self.h_grid, 
                stride=stride, 
                bias=use_bias, 
                h_crop=self.diff_size
            )
        else:
            self.c1 = eerie.nn.GConvGG(
                group, 
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=kernel_size, 
                h_grid=h_grid_crop, 
                stride=stride, 
                bias=use_bias, h_crop=True
            )

        self.c2 = eerie.nn.GConvGG(
            group, 
            in_channels=out_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            h_grid=h_grid, 
            stride=stride, 
            bias=use_bias, 
            h_crop=False
        )

        # BatchNorm Layers
        self.bn1 = torch.nn.BatchNorm2d(num_features=out_channels, eps=eps)
        self.bn2 = torch.nn.BatchNorm2d(num_features=out_channels, eps=eps)
        self.bn_out = torch.nn.BatchNorm2d(num_features=out_channels, eps=eps)

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                m.weight.data.normal_(0, torch.prod(torch.Tensor(list(m.weight.shape)[1:])) ** (-1 / 2))
                if use_bias: m.bias.data.fill_(0.0)

    def forward(self, x):
        out = torch.relu(self.bn1(self.c1(x)))
        out = self.bn2(self.c2(out))
        # shortcut
        if self.diff_size:
            out = out + x[:,:,:-(self.crop - 1),:].repeat(1, 2, 1, 1)
        elif self.override:
            out = out + x[:, :, :-(self.crop - 1), :]
        else:
            out = out + x
        out = torch.relu(self.bn_out(out))
        return out


class RRPlus_M34res(torch.nn.Module):
    def __init__(self, n_channels=48, n_classes=10, eps=2e-5, use_bias=False):
        super(RRPlus_M34res, self).__init__()

        # # G-conv approach
        group = eerie.Group('R1R+')

        # For first layer:
        N_h_RdG = 9
        base = 2
        h_grid_RdG = group.h_grid_global(N_h_RdG, base ** (N_h_RdG - 1))
        print(h_grid_RdG.grid)

        # For subsequent layers:
        N_h = 1
        base = 2
        h_grid = group.h_grid_global(N_h, base ** (N_h - 1))
        print(h_grid.grid)
        n_channels_G = int(45)  # For Nh=6 use 2.45, For Nh = 7, use 2.65, For Nh=8, use 2.85, For Nh=9, use 3.0,

        N_h_crop = 3  # <--- TODO: not sure if this is the most optimal though, but it reduces the h_axis nicely to size 1 in the last layer
        base = 2
        h_grid_crop = group.h_grid_global(N_h_crop, base ** (N_h_crop - 1))
        print(h_grid_crop.grid)

        # Conv Layers
        self.c1 = eerie.nn.GConvRdG(
            group, 
            in_channels=6, 
            out_channels=n_channels_G, 
            kernel_size=7, 
            h_grid=h_grid_RdG, 
            bias=use_bias, 
            stride=1
        )
        # ----
        first_block = []
        for i in range(3):
            override = True if i == 0 else False
            first_block.append(
                RRPlus_IdentityBlock(
                    group, 
                    in_channels=n_channels_G, 
                    out_channels=n_channels_G, 
                    kernel_size=3, 
                    stride=1, 
                    h_grid=h_grid, 
                    h_grid_crop=h_grid_crop, 
                    use_bias=use_bias, 
                    eps=eps, 
                    override=override
                )
            )
        self.first_block = nn.Sequential(*first_block)
        # ----
        sec_block = []
        for i in range(4):
            b_channels = n_channels_G if i == 0 else n_channels_G * 2
            sec_block.append(
                RRPlus_IdentityBlock(
                    group, 
                    in_channels=b_channels, 
                    out_channels=n_channels_G * 2, 
                    kernel_size=3, 
                    stride=1, 
                    h_grid=h_grid, 
                    h_grid_crop=h_grid_crop, 
                    use_bias=use_bias, 
                    eps=eps
                )
            )
        self.sec_block = nn.Sequential(*sec_block)
        # ----
        thrd_block = []
        for i in range(6):
            b_channels = n_channels_G * 2 if i == 0 else n_channels_G * 4
            thrd_block.append(
                RRPlus_IdentityBlock(
                    group, 
                    in_channels=b_channels, 
                    out_channels=n_channels_G * 4, 
                    kernel_size=3, 
                    stride=1, 
                    h_grid=h_grid, 
                    h_grid_crop=h_grid_crop, 
                    use_bias=use_bias, 
                    eps=eps
                )
            )
        self.thrd_block = nn.Sequential(*thrd_block)
        # ----
        frth_block = []
        for i in range(3):
            b_channels = n_channels_G * 4 if i == 0 else n_channels_G * 8
            frth_block.append(
                RRPlus_IdentityBlock(
                    group, 
                    in_channels=b_channels, 
                    out_channels=n_channels_G * 8, 
                    kernel_size=3, 
                    stride=1, 
                    h_grid=h_grid, 
                    h_grid_crop=h_grid_crop, 
                    use_bias=use_bias, 
                    eps=eps
                )
            )
        self.frth_block = nn.Sequential(*frth_block)
        # ----
        self.c_out = eerie.nn.GConvGG(
            group, 
            in_channels=n_channels_G * 8, 
            out_channels=n_classes, 
            kernel_size=1, 
            stride=1, 
            h_grid=h_grid, 
            bias=use_bias)

        # BatchNorm Layers
        self.bn1 = torch.nn.BatchNorm2d(num_features=n_channels_G, eps=eps)

        # Pooling
        self.pool = eerie.functional.max_pooling_R1

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                m.weight.data.normal_(0, torch.prod(torch.Tensor(list(m.weight.shape)[1:]))**(-1/2))
                if use_bias: m.bias.data.fill_(0.0)

    def forward(self, x):
        # Instead of strided conv, we use normal conv and then pooling.
        out = self.c1(x)
        out = self.pool(out, kernel_size=4, stride=1, padding=1)
        out = torch.relu(self.bn1(out))
        # print("0 ", out.shape)
        # -----
        out = self.pool(out, kernel_size=4, stride=1, padding=1)
        out = self.first_block(out)
        # print("1 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=1, padding=1)
        out = self.sec_block(out)
        # print("2 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=1, padding=1)
        out = self.thrd_block(out)
        # print("3 ", out.shape)
        out = self.pool(out, kernel_size=4, stride=4, padding=1)
        out = self.frth_block(out)
        # print("4 ", out.shape)
        # Global pooling
        out = torch.mean(out, dim=-1, keepdim=True) # pool over the time axis
        out = self.c_out(out)
        # print("5 ", out.shape)
        # Then turn into features per time point (merging scale and the channel axes)
        out = torch.max(out, dim=-2).values  # pool over the scale axis
        out = out.view(out.size(0), 10)
        return out

In [19]:
model = RRPlus_M34res(n_channels=48, n_classes=10, eps=2e-5, use_bias=False)
sum([p.numel() for p in model.parameters()])

tensor([[  1.],
        [  2.],
        [  4.],
        [  8.],
        [ 16.],
        [ 32.],
        [ 64.],
        [128.],
        [256.]])
tensor([[1.]])
tensor([[1.],
        [2.],
        [4.]])


4018860

In [20]:
x = torch.ones((2, 6, 16))
y = model(x)
y.shape

0  torch.Size([2, 45, 9, 15])
1  torch.Size([2, 45, 7, 14])
2  torch.Size([2, 90, 5, 13])
3  torch.Size([2, 180, 3, 12])
4  torch.Size([2, 360, 1, 3])
5  torch.Size([2, 10, 1, 1])


torch.Size([2, 10])

In [12]:
# x = torch.ones((2, 1, 80000))
# y = model(x)
# y.shape

# 0  torch.Size([2, 45, 9, 20000])
# 1  torch.Size([2, 45, 7, 5000])
# 2  torch.Size([2, 90, 5, 1250])
# 3  torch.Size([2, 180, 3, 312])
# 4  torch.Size([2, 360, 1, 78])
# 5  torch.Size([2, 10, 1, 1])
# torch.Size([2, 10])

0  torch.Size([2, 45, 9, 20000])
1  torch.Size([2, 45, 7, 5000])
2  torch.Size([2, 90, 5, 1250])
3  torch.Size([2, 180, 3, 312])
4  torch.Size([2, 360, 1, 78])
5  torch.Size([2, 10, 1, 1])


torch.Size([2, 10])

# efficientnet

# convnext

In [1]:
from model import *

OSError: [WinError 127] 找不到指定的程序。 Error loading "d:\anaconda3\envs\torch\lib\site-packages\torch\lib\nvfuser_codegen.dll" or one of its dependencies.

In [9]:
model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768])

NameError: name 'ConvNeXt' is not defined

In [7]:
model = convnext_tiny(pretrained=False, in_22k=False)
sum([p.numel() for p in model.parameters()])

NameError: name 'convnext_tiny' is not defined